In [1]:
pip install aind_dynamic_foraging_basic_analysis

     |████████████████████████████████| 19.1 MB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 434 kB 107.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 50.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from pathlib import Path
import glob
import json
import seaborn as sns
from datetime import datetime
from aind_dynamic_foraging_basic_analysis.licks.lick_analysis import load_nwb
import re

In [12]:
# Load behavior data

# get list of available sessions
file_pattern = "*.nwb"
nwb_folder = "/root/capsule/data/foraging_nwb_bonsai"
file_list = glob.glob(os.path.join(nwb_folder + "/" + file_pattern))
file_names = [os.path.basename(file) for file in file_list]
# file_name = file_names[20]

def parseSessionID(file_name):
    if len(re.split('[_.]', file_name)[0]) == 6:
        aniID = re.split('[_.]', file_name)[0]
        date = re.split('[_.]', file_name)[1]
        dateObj = datetime.strptime(date, "%Y-%m-%d")
    else:
        aniID = None
        dateObj = None
    
    return aniID, dateObj
    
results = [parseSessionID(file_name) for file_name in file_names]
aniIDs, dates = zip(*results)

sessionInfo = pd.DataFrame({'sessionID': file_names,
                            'aniID': aniIDs,
                            'date': dates})
#select animal
anim_name = '716325'
anim_inx = sessionInfo['aniID'] == anim_name
sessionInfo_anim = sessionInfo[anim_inx]
# sessionInfo_anim

#select date
session_date = '05-31-2024'
date_inx = sessionInfo_anim['date'] == session_date
sessionInfo_anim_date = sessionInfo_anim[date_inx]
# sessionInfo_anim_date
#get nwb
session_id = sessionInfo_anim_date['sessionID'].values[0]

nwb_folder = "/root/capsule/data/foraging_nwb_bonsai"
nwb_file = f"{nwb_folder}/{session_id}"
nwb = load_nwb(nwb_file)
nwb

/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.7.0 because version 2.6.0-alpha is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


,start_time,stop_time,animal_response,rewarded_historyL,rewarded_historyR,delay_start_time,goCue_start_time,reward_outcome_time,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,reward_random_number_right,left_valve_open_time,right_valve_open_time,block_beta,block_min,block_max,min_reward_each_block,delay_beta,delay_min,delay_max,delay_duration,ITI_beta,ITI_min,ITI_max,ITI_duration,response_duration,reward_consumption_duration,reward_delay,auto_waterL,auto_waterR,laser_on_trial,laser_wavelength,laser_location,laser_1_power,laser_2_power,laser_on_probability,laser_duration,laser_condition,laser_condition_probability,laser_start,laser_start_offset,laser_end,laser_end_offset,laser_protocol,laser_frequency,laser_rampingdown,laser_pulse_duration,session_wide_control,fraction_of_session,session_start_with,session_alternation,minimum_opto_interval,auto_train_engaged,auto_train_curriculum_name,auto_train_curriculum_version,auto_train_curriculum_schema_version,auto_train_stage,auto_train_stage_overridden,lickspout_position_x,lickspout_position_y,lickspout_position_z,reward_size_left,reward_size_right
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.892922e+06,3.892973e+06,1.0,False,True,3.892970e+06,3.892971e+06,3.892972e+06,False,True,0.8,0.5,0.9,0.678540,0.399488,0.033,0.031,20.0,20.0,35.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,15.0,15.000000,1.5,1.0,0.2,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,off,0.5,on,on,0.0,True,Uncoupled Without Baiting,2.1rwdDelay159,1.0,STAGE_FINAL,False,6757.0,12218.0,6651.5,2.0,2.0
1,3.892976e+06,3.892986e+06,1.0,False,True,3.892983e+06,3.892984e+06,3.892985e+06,False,True,0.8,0.5,0.9,0.721318,0.547727,0.033,0.031,20.0,20.0,35.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,15.0,7.199677,1.5,1.0,0.2,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,off,0.5,on,on,0.0,True,Uncoupled Without Baiting,2.1rwdDelay159,1.0,STAGE_FINAL,False,6757.0,12218.0,6651.5,2.0,2.0
2,3.892986e+06,3.893002e+06,1.0,False,True,3.892989e+06,3.893001e+06,3.893001e+06,True,True,0.8,0.5,0.9,0.484397,0.724972,0.033,0.031,20.0,20.0,35.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,15.0,2.942915,1.5,1.0,0.2,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,off,0.5,on,on,0.0,True,Uncoupled Without Baiting,2.1rwdDelay159,1.0,STAGE_FINAL,False,6757.0,12218.0,6651.5,2.0,2.0
3,3.893002e+06,3.893036e+06,1.0,False,True,3.893005e+06,3.893035e+06,3.893035e+06,False,True,0.8,0.5,0.9,0.790330,0.347795,0.033,0.031,20.0,20.0,35.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,15.0,3.354985,1.5,1.0,0.2,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,off,0.5,on,on,0.0,True,Uncoupled Without Baiting,2.1rwdDelay159,1.0,STAGE_FINAL,False,6756.5,12218.0,6651.5,2.0,2.0
,start_time,stop_time,animal_response,rewarded_historyL,rewarded_historyR,delay_start_time,goCue_start_time,reward_outcome_time,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,reward_random_number_right,left_valve_open_time,right_valve_open_time,block_beta,block_min,block_max,min_reward_each_block,delay_beta,delay_min,delay_max,delay_duration,ITI_beta,ITI_min,ITI_max,ITI_duration,response_duration,reward_consumption_duration,reward_delay,auto_waterL,auto_waterR,laser_on_trial,laser_wavelength,laser_location,laser_1_power,laser_2_power,laser_on_probability,laser_duration,laser_condition,laser_condition_probability,laser_start,laser_start_offset,laser_end,laser_end_offset,laser_protocol,laser_frequency,laser_rampingdown,laser_pulse_duration,session_wide_control,fraction_of_session,session_start_with,session_alternation,minimum_opto_interval,auto_train_engaged,auto_train_curriculum_name,auto_train_curriculum_version,auto_train_curriculum_schema_version,auto_train_stage,auto_train_stage_overridden,lickspout_position_x,lickspout_position_y,lickspout_position_z,reward_size_left,reward_size_right
id

,sessionID,aniID,date
2599,716325_2024-05-31_10-31-14.nwb,716325,2024-05-31
